In [ ]:
import suspect
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
#basis = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/LCModel_basis/7T_SE_TE9_mac.basis'
basis = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/NewBasis/7T_SPECIAL_real_TE9_NoMM.basis'


In [ ]:
fullspecs = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/IMNS/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*subtracted.raw')
fullspecs

In [ ]:
water = [i.replace('/IMNS','/suspect_preprocessing').replace('spec-lr-special_subtracted.raw', 'spec-lr-special_water_woOVSavg.raw') for i in fullspecs]

In [ ]:
[os.path.isfile(i) for i in water]

In [ ]:
# Copy file to derivatives Directory

In [ ]:
import shutil

In [ ]:
for i in water:
    outname = i.replace('derivatives/suspect_preprocessing', 'derivatives/IMNS')
    os.makedirs(os.path.dirname(outname), exist_ok=True)
    shutil.copyfile(i, outname)

In [ ]:
scm_control = '''
$LCMODL
title='{sub}_{roi}'
atth2o= 0.8111
key = 210387309
deltat= 1.250e-04
doecc= F
dows= T
hzpppm= 2.9721e+02
neach= 999
nunfil= 4096
SDDEGZ = 1
SDDEGP = 0
shifmn = -0.05
shifmx = 0.05
FIXSHF = T
ppmend= 1.1
ppmst= 4.2
wconc= 43432
dkntmn= 0.25
echot= 9
nomit= 12
chomit= '-CrCH2' 'Gua' 'Ser' 'Lip13a' 'Lip13b' 'Lip09' 'MM09' 'Lip20' 'MM20' 'MM12' 'MM14' 'MM17'
NRATIO = 0
FILBAS='{basis}'
FILRAW='{filename}.raw'
LTABLE=7, FILTAB='{filename}/{filename}.table'
FILPS='{filename}.ps'
LCSV=11, FILCSV='{filename}/{filename}.csv'
LCOORD=9, FILCOO='{filename}/{filename}.coord'
LPRINT=6, FILPRI='{filename}/{filename}.print'
LCORAW=10, FILCOR='{filename}/{filename}.coraw'
filh2o='{water}'
$END'''

In [ ]:
for n,i in enumerate(fullspecs):
    control_name = i.replace('.raw', '_IMNS.control')
    print(control_name)
    file_name = os.path.basename(i).rstrip('.raw')
    sub=file_name.split('_')[0]
    roi=file_name.split('_')[2]
    print(file_name)
    with open(control_name, 'w') as ctrl:
        ctrl.write(scm_control.format(basis=basis, 
                                      filename=file_name,
                                     water=water[n],
                                      sub=sub,
                                      roi=roi))

In [ ]:
import subprocess

In [ ]:
import sys
sys.path.append('/home/orco/data/MacroMols/MRS/MRSpype')
from mrspype.utils import utils

In [ ]:
spec_ctrls = '/home/orco/data/MacroMols/MRS/MacroMols_7T/derivatives/IMNS/sub-*/mrs/sub-*_nuc-1H_loc-*_spec-lr-special*IMNS.control'

In [ ]:
spec_ctrls = glob.glob(spec_ctrls)

In [ ]:
spec_ctrls

In [ ]:
len(spec_ctrls)

In [ ]:
curr_dir = os.getcwd()
curr_dir

In [ ]:
# for i in spec_ctrls:
#     dirname = os.path.dirname(i)
#     os.chdir(dirname)
#     #print(os.getcwd())
#     os.makedirs(i.replace('_SCM.control',''), exist_ok=True)
#     proc = subprocess.run('/home/orco/.lcmodel/bin/lcmodel < {filename}'.format(filename=os.path.basename(i)), shell=True, check=True, capture_output=True)
#     print(proc)

In [ ]:
from joblib import Parallel, delayed

def lcm(i):
    dirname = os.path.dirname(i)
    os.chdir(dirname)
    #print(os.getcwd())
    os.makedirs(i.replace('_IMNS.control',''), exist_ok=True)
    proc = subprocess.run('/home/orco/.lcmodel/bin/lcmodel < {filename}'.format(filename=os.path.basename(i)), shell=True, check=True, capture_output=True)
    print(proc)
    
Parallel(n_jobs=32)(delayed(lcm)(i) for i in spec_ctrls)